# Functional programming in Python (operator, functional, itertoools, toolz)

- Pure functions
- Recursive functions
- Anonymous functions
- Lazy evaluation
- Higher-order functions
- Decorators
- Partial application
- Using `operator`
- Using `functional`
- Using `itertools`
- Pipelines with `toolz`

In [ ]:
import numpy as np

## Pure functions

### Deterministic

Pure

In [ ]:
np.exp(5), np.exp(5)

Not pure

In [ ]:
np.random.randn(), np.random.randn()

### No side effects

Pure

In [ ]:
def f(xs):
    '''Modify value at first index.'''
    if len(xs) > 0:
        xs = list(xs)
        xs[0] = '@'
    return xs

In [ ]:
xs = [1,2,3]
f(xs), xs

Not pure

In [ ]:
def g(xs):
    '''Modify value at first index.'''
    if len(xs) > 0:
        xs[0] = '@'
    return xs

In [ ]:
xs = [1,2,3]
g(xs), xs

### Exercise

Is the function `h` pure or impure?

In [ ]:
def h(n, xs=[]):
    for i in range(n):
        xs.append(i)
    return xs

The function is not deterministic, and it has side effects!

In [ ]:
n = 5
xs = [1,2,3]

Non-deterministic

In [ ]:
h(n)

In [ ]:
h(n)

To avoid non-determinism, do not set default mutable arguments. The usaal Python idiom is

```python
def func(xs=None):
    """Docstring."""
    
    if xs is None:
        xs = []
    do_something(xs)
```

Side effects

In [ ]:
xs = [1,2,3]

In [ ]:
h(n, xs)

In [ ]:
xs

## Recursive functions

A recursive function is one that calls itself. Python supports recursion, but recursive functions in Python are not efficient and iterative algorithms are preferred in general.

In [ ]:
def rec_sum(xs):
    """Recursive sum."""
    
    if len(xs) == 0:
        return 0
    else:
        return xs[0] + rec_sum(xs[1:])

In [ ]:
rec_sum([1,2,3,4])

## Anonymous functions

In [ ]:
lambda x, y: x + y

In [ ]:
add = lambda x, y: x + y

In [ ]:
add(3, 4)

In [ ]:
lambda x, y: x if x < y else y

In [ ]:
smaller = lambda x, y: x if x < y else y

In [ ]:
smaller(9,1)

## Lazy evaluation

In [ ]:
range(10)

In [ ]:
list(range(10))

### Generators

Generators behave like functions that retain the last state and can be re-entered. Results are returned with `yield` rather than `return`. Generators are used extensively in Python, and almost exclusively in the `itertools` and `toolz` packages we will review later in this  notebook.

#### Differences between a function and a generator

In [ ]:
def fib_eager(n):
    """Eager Fibonacci function."""
    
    xs = []
    a, b = 1,1
    for i in range(n):
        xs.append(a)
        a, b = b, a + b
    return xs

In [ ]:
fib_eager(10)

In [ ]:
def fib_lazy(n):
    """Lazy Fibonacci generator."""
    
    a, b = 1,1
    for i in range(n):
        yield a
        a, b = b, a + b

In [ ]:
fib_lazy(10)

In [ ]:
list(fib_lazy(10))

In [ ]:
fibs1 = fib_eager(10)

In [ ]:
for i in fibs1:
    print(i, end=',')

In [ ]:
for i in fibs1:
    print(i, end=',')

In [ ]:
fibs2 = fib_lazy(10)

In [ ]:
for i in fibs2:
    print(i, end=',')

In [ ]:
for i in fibs2:
    print(i, end=',')

#### Generators can return infinite sequences

In [ ]:
def iota(start = 1):
    """An infinite incrementing genrator."""
    
    while True:
        yield start
        start += 1    

In [ ]:
for i in iota():
    print(i, end=',')
    if i > 25:
        break

## Higher order functions

- Take a function as an argument 
- Returns a function

In [ ]:
def dist(x, y, measure):
    """Returns distance between x and y using given measure.
    
    measure is a function that takes two arguments x and y.
    """
    
    return measure(x, y)

In [ ]:
def euclid(x, y):
    """Returns Euclidean distance between x and y."""
    
    return np.sqrt(np.sum(x**2 + y**2))

In [ ]:
x = np.array([0,0])
y = np.array([3,4])

dist(x, y, measure=euclid)

### Standard HOFs

In [ ]:
from functools import reduce

In [ ]:
list(map(lambda x: x**2, range(10)))

In [ ]:
list(filter(lambda x: x % 2 == 0, range(10)))

In [ ]:
reduce(lambda x, y: x + y, range(10))

In [ ]:
reduce(lambda x, y: x + y, range(10), 10)

## Example: Flattening a nested list

In [ ]:
s1 = 'the quick brown fox'
s2 = 'jumps over the dog'
xs = [s.split() for s in [s1, s2]]
xs

Using a nested for loop

In [ ]:
ys = []
for x in xs:
    for y in x:
        ys.append(y)
ys

Using a list comprehension

In [ ]:
[y for x in xs for y in x]

Using `reduce`

In [ ]:
reduce(lambda x, y: x + y, xs)

## Closure

In [ ]:
def f(a):
    """The argument given to f is visible in g when g is called."""
    
    def g(b):
        return a + b
    return g

In [ ]:
g3 = f(3)
g5 = f(5)

In [ ]:
g3(4), g5(4)

## Decorators

A decorator is a higher-order function that takes a function as input and returns a *decorated* version of the original function with additional capabilities. There is syntactical sugar for decorators in Python using the `@decorator_function` notation as shown below.

In [ ]:
import time

In [ ]:
def timer(f):
    """Times how long f takes."""
    
    def g(*args, **kwargs):
        
        start = time.time()     
        res = f(*args, **kwargs)
        elapsed = time.time() - start
        return res, elapsed
    return g

In [ ]:
def slow(n=1):
    time.sleep(n)
    return n

Use as a function

In [ ]:
slow_t1 = timer(slow)

In [ ]:
slow_t1(0.5)

Use as a decorator

In [ ]:
@timer
def slow_t2(n=1):
    time.sleep(n)
    return n

In [ ]:
slow_t2(0.5)

## Partial application

Partial application takes a function with two or more parameters, and returns the function with some parameters *filled in*. Partial application is very useful when constructing pipelines that transform data in stages.

In [ ]:
from functools import partial

In [ ]:
def add(a, b):
    """Add a and b."""
    
    return a + b

In [ ]:
list(map(partial(add, b=10), range(5)))

## Using `operator`

Operator provides named version of Python operators, and is typically used in place of anonymous functions in higher order functions to improve readability.

In [ ]:
import operator as op

In [ ]:
xs = [('a', 3), ('b', 1), ('c', 2)]

In [ ]:
sorted(xs, key=op.itemgetter(1))

In [ ]:
sorted(xs, key=lambda x: x[1])

In [ ]:
reduce(op.add, range(1,5))

In [ ]:
reduce(lambda a, b: a + b, range(1,5))

## Using `functional`

We have already seen the use of `reduce` and `partial` from `functools`. Another useful function from the package is the `lrucache` decorator.

In [ ]:
def fib(n):
    """Recursive version of Fibonacci."""
    
    print('Call fib(%d)' % n)
    
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-2) + fib(n-1)

Notice the inefficiency from repeatedly calling the function with the same arguments.

In [ ]:
fib(6)

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache(maxsize=None)
def fib_cache(n):
    """Recursive version of Fibonacci."""
    
    print('Call fib(%d)' % n)
    
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib_cache(n-2) + fib_cache(n-1)

In [ ]:
fib_cache(6)

## Using `itertools`

The `itertools` package provides tools for efficient looping.

In [ ]:
import itertools as it

### Generators

In [ ]:
list(it.islice(it.count(), 10))

In [ ]:
list(it.islice(it.cycle([1,2,3]), 10))

In [ ]:
list(it.islice(it.repeat([1,2,3]), 10))

In [ ]:
list(it.zip_longest(range(5), 'abc'))

In [ ]:
list(it.zip_longest(range(5), 'abc', fillvalue='out of donuts'))

### Permutations and combinations

In [ ]:
list(it.permutations('abc'))

In [ ]:
list(it.permutations('abc', 2))

In [ ]:
list(it.combinations('abc', 2))

In [ ]:
list(it.combinations_with_replacement('abc', 2))

In [ ]:
list(it.product('abc', repeat=2))

### Miscellaneous

In [ ]:
list(it.chain([1,2,3], [4,5,6], [7,8,9]))

In [ ]:
list(it.chain.from_iterable([[1,2,3], [4,5,6], [7,8,9]]))

In [ ]:
nums = [1,3,5,7,2,4,6,1,3,5,7,9,2,2,2]

In [ ]:
for i, g in it.groupby(nums, key=lambda x: x % 2 ==0):
    print(i, list(g))

In [ ]:
list(it.takewhile(lambda x: x % 2 == 1, nums))

In [ ]:
list(it.dropwhile(lambda x: x % 2 == 1, nums))

## Using `toolz`

The `toolz` package provides a very rich set of functional operators, and is recommended if you want to program in the functional style using Python.

In [ ]:
import toolz

In [ ]:
from toolz import sliding_window, pipe, frequencies, concat

In [ ]:
import toolz.curried as c

Example: Read in some documents, remove punctuation, split into words and then into individual characters, and find the most commonly occurring sliding windows containing 3 characters.

This is most naturally done by piping in a data set through a series of transforms.

In [ ]:
d1 = 'the doo doo doo the dah dah dah'
d2 = 'every breath she takes, every move she makes'
d3 = 'another brick in the wall'
d4 = 'another one bites the dust and another one gone'
docs = [d1,d2,d3,d4]

In [ ]:
import string

In [ ]:
triple_freqs = pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    concat,
    c.sliding_window(3),
    frequencies,
)

In [ ]:
sorted(triple_freqs.items(), key=lambda x: x[1], reverse=True)[:5]

### Step by step

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    concat,
    c.sliding_window(3),
    frequencies,
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    concat,
    c.sliding_window(3),
    frequencies,
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    concat,
    c.sliding_window(3),
    frequencies,
)

In [ ]:
pipe(
    docs,
    list
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    list
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    list
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    list
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    concat,
    list
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    concat,
    c.sliding_window(3),
    list
)

In [ ]:
pipe(
    docs,
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    concat,
    concat,
    c.sliding_window(3),
    frequencies
)